## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
from pathlib import Path
from pprint import pprint
import pandas as pd
import requests
import gmaps
import json

# Load key
resources = Path("resources")
with open(resources / "keys.json", "r") as keys:
    g_key = json.load(keys)[0]["google"]

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv", index_col=[0])
city_data_df.head()

,City,Country,Lat,Lon,MaxTemp,MinTemp,Humidity,Cloudniness,Wind Speed,Current Description
0,Ushuaia,AR,-54.8000,-68.3000,33.46,33.46,93,40,4.61,scattered clouds
1,Jamestown,US,42.0970,-79.2353,71.19,65.95,66,0,6.91,clear sky
2,Atuona,PF,-9.8000,-139.0333,76.93,76.93,76,18,23.67,few clouds
3,Hobart,AU,-42.8794,147.3294,53.26,49.86,76,20,4.61,few clouds
4,Chokurdakh,RU,70.6333,147.9167,53.47,53.47,56,73,7.56,broken clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria

def get_temp(key):
    while True:
        temp = input(f"Please input the {key} temperature criteria(Farenheit)\n")
        try:
            temp = float(temp)
            print(f"Thank you, {key} temperature is: {temp}")
            return temp
        except:
            print("Please enter a valid float or integer number!")

print("-"*20)
min_temp = get_temp("minimum")
max_temp = get_temp("maximum")

--------------------
Thank you, minimum temperature is: 50.0
Thank you, maximum temperature is: 70.0


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df = city_data_df.loc[(
    city_data_df["MinTemp"] > min_temp) &
    (city_data_df["MaxTemp"] < max_temp)
]
city_data_df.head()

,City,Country,Lat,Lon,MaxTemp,MinTemp,Humidity,Cloudniness,Wind Speed,Current Description
4,Chokurdakh,RU,70.6333,147.9167,53.47,53.47,56,73,7.56,broken clouds
10,Tuktoyaktuk,CA,69.4541,-133.0374,53.60,53.60,82,75,12.66,broken clouds
13,Busselton,AU,-33.6500,115.3333,53.82,53.82,78,0,15.46,clear sky
14,Sitka,US,57.0531,-135.3300,67.89,67.89,68,100,5.75,overcast clouds
17,Nikolskoye,RU,59.7035,30.7861,64.29,58.78,83,100,6.76,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
missing = city_data_df.isna()
missing.shape

(614, 10)

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.dropna()
clean_df.head()

,City,Country,Lat,Lon,MaxTemp,MinTemp,Humidity,Cloudniness,Wind Speed,Current Description
4,Chokurdakh,RU,70.6333,147.9167,53.47,53.47,56,73,7.56,broken clouds
10,Tuktoyaktuk,CA,69.4541,-133.0374,53.60,53.60,82,75,12.66,broken clouds
13,Busselton,AU,-33.6500,115.3333,53.82,53.82,78,0,15.46,clear sky
14,Sitka,US,57.0531,-135.3300,67.89,67.89,68,100,5.75,overcast clouds
17,Nikolskoye,RU,59.7035,30.7861,64.29,58.78,83,100,6.76,overcast clouds


In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "MaxTemp", "Current Description", "Lat", "Lon"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,MaxTemp,Current Description,Lat,Lon,Hotel Name
4,Chokurdakh,RU,53.47,broken clouds,70.6333,147.9167,
10,Tuktoyaktuk,CA,53.60,broken clouds,69.4541,-133.0374,
13,Busselton,AU,53.82,clear sky,-33.6500,115.3333,
14,Sitka,US,67.89,overcast clouds,57.0531,-135.3300,
17,Nikolskoye,RU,64.29,overcast clouds,59.7035,30.7861,
23,Severo-Kuril'sk,RU,57.24,overcast clouds,50.6789,156.1250,
26,Busselton,AU,53.82,clear sky,-33.6500,115.3333,
39,Geraldton,AU,55.81,clear sky,-28.7667,114.6000,
44,Margate,GB,67.69,scattered clouds,51.3813,1.3862,
46,Poli,TZ,55.29,overcast clouds,-3.3333,36.8000,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
base_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
# 6b. Iterate through the hotel DataFrame.
for index in hotel_df.index:
    # 6c. Get latitude and longitude from DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lon"]
    params["location"] = f"{lat}, {lon}"
    params["keyword"] = "Hotel"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # 6e. Make request and retrieve the JSON data from the search. 
    r = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    if len(r["results"]) > 0:
        hotel_df.loc[index, "Hotel Name"] = r["results"][0]["name"]

hotel_df.head()

,City,Country,MaxTemp,Current Description,Lat,Lon,Hotel Name
4,Chokurdakh,RU,53.47,broken clouds,70.6333,147.9167,
10,Tuktoyaktuk,CA,53.60,broken clouds,69.4541,-133.0374,Smitty's Bed and Breakfast
13,Busselton,AU,53.82,clear sky,-33.6500,115.3333,Esplanade Hotel
14,Sitka,US,67.89,overcast clouds,57.0531,-135.3300,Sitka Hotel
17,Nikolskoye,RU,64.29,overcast clouds,59.7035,30.7861,Tourist House - Sablino


In [42]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head()

,City,Country,MaxTemp,Current Description,Lat,Lon,Hotel Name
10,Tuktoyaktuk,CA,53.60,broken clouds,69.4541,-133.0374,Smitty's Bed and Breakfast
13,Busselton,AU,53.82,clear sky,-33.6500,115.3333,Esplanade Hotel
14,Sitka,US,67.89,overcast clouds,57.0531,-135.3300,Sitka Hotel
17,Nikolskoye,RU,64.29,overcast clouds,59.7035,30.7861,Tourist House - Sablino
26,Busselton,AU,53.82,clear sky,-33.6500,115.3333,Esplanade Hotel


In [7]:
# 8a. Create the output File (CSV)
output_data_file = Path("Vacation_Search") / "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df_check = pd.read_csv(output_data_file, index_col="City_ID")
clean_hotel_df_check.head()

,City,Country,MaxTemp,Current Description,Lat,Lon,Hotel Name
City_ID,,,,,,,
10,Tuktoyaktuk,CA,53.60,broken clouds,69.4541,-133.0374,Smitty's Bed and Breakfast
13,Busselton,AU,53.82,clear sky,-33.6500,115.3333,Esplanade Hotel
14,Sitka,US,67.89,overcast clouds,57.0531,-135.3300,Sitka Hotel
17,Nikolskoye,RU,64.29,overcast clouds,59.7035,30.7861,Tourist House - Sablino
26,Busselton,AU,53.82,clear sky,-33.6500,115.3333,Esplanade Hotel


In [8]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}, {MaxTemp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lon"]]
locations.head()

,Lat,Lon
City_ID,,
10,69.4541,-133.0374
13,-33.6500,115.3333
14,57.0531,-135.3300
17,59.7035,30.7861
26,-33.6500,115.3333


In [9]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["MaxTemp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))